# Navigation

---

### 1. Prepare dependencies and environment

Take a look at README.md before executing this notebook and make sure that the kernel is set to **p1_navigation**. The environment for this project is [Banana](https://github.com/kotogasy/unity-ml-banana) from Unity, and it's provided in the `setup` folder.

In [2]:
!pip -q install ./setup

import sys

import numpy as np
from numpy_ringbuffer import RingBuffer

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch import nn

from setup import unityagents
from unityagents import UnityEnvironment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Unity environments contain **brains**, our interfaces for controlling agents. We'll be conrtolling the first (default) brain in the environment. It's also useful to keep information such as `state_size` and `action_size`.

In [3]:
env = UnityEnvironment(file_name="setup/Banana.app")

# use the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]
action_size = brain.vector_action_space_size
state = env_info.vector_observations[0]
state_size = len(state)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


### 2. Replay buffer

We'll use the **uniform** variation of the replay buffer here, meaning that all stored tuples have the same chance of being selected for replay. Nonetheless, we can provide some additional methods and data, such as `max_priority()` and `update_priority()`, that will help us write a more abstract agent, in case we want to try the prioritized variant.

Every tuple is stored as `(s, a, r, ns, d, p)` where:

- `s` is the state at the beginning of the timestep
- `a` is the action that was taken
- `r` is the reward obtained in the next timestep
- `ns` is the state at the next timestep (we'll refer to this as $s'$ as well)
- `d` is a boolean value that determines if the episode ended
- `p` is the **priority** of the tuple, ignored in this case and kept here because it will make the transition to the prioritized version easier. At the moment, however, every tuple will be considered with `p = max_priority() = 0` (not stored), _regardless of p_.

When sampling a batch of `n` tuples, we'll obtain a single tuple `([s], [a], [r], [ns], [d], [w])` where:
- `[s]`, `[a]`, `[r]`, `[ns]`, `[d]` are **torch tensors** with `n` rows. Keep in mind that `[s]` and `[ns]` can have more than one column, depending on the `state_size`.
- `[w]` is the `n`-rows tensor of **importance sampling weights**. Since this is the uniform case, every tuple has the weight `1/n`. Again, this is done because it will make the transition to the prioritized version easier.

In [4]:
# code is commented in ReplayBuffers/UniformReplayBuffer.py
class UniformReplayBuffer():

    def __init__(self, size):
        self.buff = RingBuffer(capacity=size, dtype=object)
    
    def sample(self, n, replace=True):
        samples = np.random.choice(np.array(self.buff), n, replace)
        
        s = torch.FloatTensor([sample[0] for sample in samples])
        a = torch.LongTensor([sample[1] for sample in samples])
        r = torch.FloatTensor([sample[2] for sample in samples])
        ns = torch.FloatTensor([sample[3] for sample in samples])
        d = torch.FloatTensor([sample[4] for sample in samples])
        w = torch.ones(d.size()) / n
        
        return s, a, r, ns, d, w
    
    def add(self, observation):
        s, a, r, ns, d, _ = observation
        self.buff.append((s, a, r, ns, d))
    
    def size(self):
        return len(self.buff)
    
    def max_priority(self):
        return 0
    
    def update_priority(self, observations):
        pass

### 3. Q Network

We have two choices for this network's architecture:

- `DQN`, the original Deep Q-Network
- `DuelingDQN`, the Dueling Deep Q-Network

Both models receive the current state $s$ as input and provide a vector for all estimated $Q(s, a)$ values at once.

#### 3.1 DQN

The `DQN` class is straightforward and uses less parameters, since it only uses one stream of fully connected layers:

- `state_size` $\to 64$ followed by `relu` activations
- $64 \to 128$ and `relu`
- $128 \to 64$ and `relu`
- $64 \to Q(s, a)$, that is $64 \to$ `action_size` and no activation

In [5]:
class DQN(nn.Module):
    
    def __init__(self, state_size, action_size, hidden_layers=[64, 128, 64]):
        super(DQN, self).__init__()
        
        self.hidden_layers = nn.ModuleList([nn.Linear(state_size, hidden_layers[0])])
        
        A = hidden_layers[:-1]  # -> [64, 128] (using the predefined values)
        B = hidden_layers[1:]   # -> [128, 64]
        in_out = zip(A, B)      # -> [(64, 128), (128, 64)]
        self.hidden_layers.extend([nn.Linear(a, b) for a, b in in_out])
        
        self.output_layer = nn.Linear(hidden_layers[-1], action_size)

    def forward(self, state):
        for layer in self.hidden_layers:
            state = layer(state)
            state = F.relu(state)
        state = self.output_layer(state)
        return state

#### 3.2 DuelingDQN

On the other hand, `DuelingDQN` has two streams of independent layers: the **value** of a state $V(s)$ (scalar) and the **advantage** of taking an action $A(s, a)$ (same shape as the output), related by $A(s, a) = Q(s, a) - V(s)$. The idea is that, in many scenarios, the value of a state won't depend that much on the action the agent takes, so decoupling these values can help make better estimates.

So, the class `DuelingDQN` uses two identical streams of independent fully connected layers:

- `state_size` $\to 64$ and `relu`
- $64 \to 128$ and `relu`
- $128 \to 64$ and `relu`

The streams end in:

- $64 \to A(s, a)$, that is $64 \to$ `action_size` and no activation
- $64 \to V(s)$, that is $64 \to 1$ and no activation

To merge them, however, we can't simply add $V$ and $A$ as $Q(s, a) = V(s) + A(s, a)$, because $A$ and $V$ **won't be identifiable**. We can, instead, write

$$Q(s, a) = V(s) + A(s, a) - \max_{a'} A(S, a')$$

so that when the best action $a^*$ is selected, we get that $Q(s, a^*) = V(s)$, something that is not guaranteed otherwise. To further improve this, we can substitute the $\max$ operator for a mean of the available actions. This is the final equation used in the `DuelingDQN` class, where $|\mathbb{A}(s)| =$ `action_size`.

$$Q(s, a) = V(s) + A(s, a) - \frac{1}{|\mathbb{A}(s)|} \sum_{a' \in \mathbb{A}(s)} A(s, a')$$

In [12]:
class DuelingDQN(nn.Module):
    pass

### 4. Error estimation

Since this is a bootstrapping method, when we're calculating the **temporal-difference** error $\delta_t$ we'll have to rely on an estimate of future returns. Two proposed variants for this are fixed Q-targets and Double DQN.

The functions `dt_dqn`, `dt_double_dqn` implement these two variants. Both take an extra boolean parameter `d` that signals when the episode is over, since in that case, there is no future return and the TD error is simply $\delta_t = r - Q(s, a)$.

Note that these functions expect `s`, `a`, `r`, `ns` and `d` as **torch tensors**, where the number of rows indicates the batch size. The return value is also a vector with the same number of rows, one for each tuple.

#### 4.1 Fixed Q-targets

The original paper proposed the method of fixed Q-targets, essentially **duplicating the network**, creating $Q_{local}$ and $Q_{target}$, where the former is used to determine the policy and the latter is updated less frequently and only used to estimate future returns. In particular, the equation for $\delta_t$ is

$$\delta_t(s, a, r, s') = r + \gamma \max_{a'} Q_{target}(s', a') - Q_{local}(s, a)$$

In [6]:
def dt_dqn(s, a, r, ns, d, q_local, q_target, gamma):
    with torch.no_grad():   # no need for gradients when we're evaluating the TD target
        QT = q_target(ns)   # evaluate the next state using the target network (out: [n * action_size])
        QT = QT.max(1)      # take the max along the column (actions) (out: two tensors [n])
        QT = QT[0]          # - [0] has the max values for each element in the batch, 
                            # - [1] has the indexes of the max values

    a = a.unsqueeze(1)      # reshape [n] -> [n * 1]
    QL = q_local(s)         # evaluate the current state using the local network (out: [n * action_size])
    QL = QL.gather(1, a)    # for each row, take the column in QL indicated by a (out: [n * 1])
    QL = QL.squeeze(1)      # reshape [n * 1] -> [n]

    return r + gamma * QT * (1 - torch.FloatTensor(d)) - QL

#### 4.2 Double DQN

This is largely based on fixed Q-targets, since it also uses $Q_{local}$ and $Q_{target}$. We can rewrite $\max Q$ using $\arg \max$ to break down the task of **choosing** and **evaluating** an action in two different steps.

$$\max_{a'} Q_{target}(s', a') = Q_{target}(s', \arg \max_{a'} Q_{target}(s', a'))$$

This can lead to the **overestimation** of targets. Simply put, when taking the $\max$ among noisy numbers (which is especially true in the beginning) we're likely to pick the action where the approximator adds more positive noise. Separating the task of picking and evaluating the action to two different approximators may decrease this issue, since both networks now have to "agree" on the outcome of an action.

The original estimate can be improved using the one provided in the Double DQN paper. The proposed solution is to use $Q_{local}$ for the choice and $Q_{target}$ for the evaluation as follows

$$
Q_{target}(s', \arg \max_{a'} Q_{local}(s', a'))\\
\implies \delta_t(s, a, r, s') = r + \gamma Q_{target}(s', \arg \max_{a'} Q_{local}(s', a')) - Q_{local}(s, a)
$$

In [7]:
def dt_double_dqn(s, a, r, ns, d, q_local, q_target, gamma):
    with torch.no_grad():         # no need for gradients when we're evaluating the TD target
        QLns = q_local(ns)        # evaluate the next state using the local network (out: [n * action_size])
        QLns = QLns.max(1)        # take the max along the column (actions) (out: two tensors [n])
        QLns = QLns[1]            # [1] has the indexes of the max values
        QLns = QLns.unsqueeze(1)  # reshape [n] -> [n * 1]

        QT = q_target(ns)         # evaluate the next state using the target network (out: [n * action_size])
        QT = QT.gather(1, QLns)   # for each row, take the value estimated by the target network for
                                  # the best action estimated by the local network (out: [n * 1])
        QT = QT.squeeze(1)        # reshape [n * 1] -> [n]

    a = a.unsqueeze(1)            # reshape [n] -> [n * 1]
    QL = q_local(s)               # evaluate the current state using the local network (out: [n * action_size])
    QL = QL.gather(1, a)          # for each row, take the column in QL indicated by a (out: [n * 1])
    QL = QL.squeeze(1)            # reshape [n * 1] -> [n]

    return r + gamma * QT * (1 - torch.FloatTensor(d)) - QL

### 5. Agent

Let's put the pieces together.

In [8]:
class QNetworkAgent():
    
    def __init__(self, QNetwork, state_size, action_size, 
                 replay_buffer, Delta, 
                 eps=1, eps_decay=0.9995, min_eps=0.0001, gamma=0.99, 
                 alpha=0.001, tau=0.01,
                 update_every=4, batch_size=64, learning=True):
        self.state_size, self.action_size = state_size, action_size
        self.q_local, self.q_target = QNetwork(state_size, action_size), QNetwork(state_size, action_size)
        self.q_target.eval()
        self.replay_buffer = replay_buffer
        self.Delta = Delta
        self.optimizer = optim.Adam(self.q_local.parameters(), lr=alpha)
        self.learning = learning
        self.eps, self.eps_decay, self.min_eps = eps, eps_decay, min_eps
        self.gamma, self.alpha, self.tau = gamma, alpha, tau
        self.update_every, self.update_i, self.batch_size = update_every, 0, batch_size
        
    def act(self, s):
        if not self.learning or np.random.uniform() > self.eps:
            with torch.no_grad():
                s = torch.FloatTensor(s).unsqueeze(0)
                return int(self.q_local(s).max(1)[1])
        else:
            return np.random.randint(self.action_size)
    
    def store(self, s, a, r, ns, d):
        p = self.replay_buffer.max_priority()
        self.replay_buffer.add((s, a, r, ns, d, p))
        if self.update_i == 0 and self.replay_buffer.size() >= self.batch_size:
            self.learn()
        self.update_i = (self.update_i + 1) % self.update_every
        self.eps = max(self.eps * self.eps_decay, self.min_eps)
    
    def learn(self):
        s, a, r, ns, d, w = self.replay_buffer.sample(self.batch_size)
        td_delta = self.Delta(s, a, r, ns, d, self.q_local, self.q_target, self.gamma)
        self.replay_buffer.update_priority(zip(s, a, r, ns, d, torch.abs(td_delta)))
                
        self.optimizer.zero_grad()
        loss = torch.sum(w * (td_delta ** 2))  # weighted mse
        loss.backward()
        self.optimizer.step()
        
        with torch.no_grad():
            for local, target in zip(self.q_local.parameters(), self.q_target.parameters()):
                target.copy_(target + self.tau * (local - target))

In [9]:
agent = QNetworkAgent(
    DQN,
    state_size, action_size,
    UniformReplayBuffer(100_000),
    dt_double_dqn
)

In [11]:
# last100 = []
for i in range(355, 1001):
    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.vector_observations[0]
    score = 0
    done = False
    while not done:
        action = agent.act(state)

        env_info = env.step(action)[brain_name]
        next_state = env_info.vector_observations[0]
        reward = env_info.rewards[0]
        done = env_info.local_done[0]

        agent.store(state, action, reward, next_state, done)
        score += reward
        state = next_state

    # if i % 10 == 0:
    
    last100.append(score)
    if len(last100) > 100:
        last100.pop(0)
    
    average = np.mean(last100)
    print("\rEpisode: {}, Score: {}, Average: {}".format(i, score, average), end="")
    sys.stdout.flush()

    """
    if average > 13:
        print("\rSolved in {} episodes. Average: {}".format(i, average), end="")
        sys.stdout.flush()
        break
    """
    
# print("Score: {}".format(score))

Episode: 1000, Score: 11.0, Average: 13.11

In [19]:
# torch.save(agent.q_local.state_dict(), 'checkpoint.pth')
# agent.q_local.load_state_dict(torch.load('checkpoint.pth', map_location='cpu'))

In [12]:
env_info = env.reset(train_mode=False)[brain_name]
agent.learning = False
state = env_info.vector_observations[0]
score = 0
done = False
while not done:
    action = agent.act(state)

    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]

    agent.store(state, action, reward, next_state, done)
    score += reward
    state = next_state
print(score)

13.0


In [13]:
env.close()